# Technical Report: CSCI Independent Study
2023 Spring Semester

Student: Yufeng Wu

Advisor: Rohit

## I. Proposed Research Contribution

We propose a new method that utilizes information derived from causal Directed Acyclic Graphs (DAGs) to identify the stable components of a predictive model when faced with distribution shifts. This method enables us to selectively retrain only the necessary parts of the model using data from the altered environment. Our primary objective is to outperform the baseline model in the shifted environment, surpassing the results that would be achieved by training a completely new model from scratch using the limited data available from the shifted environment.


## II. Problem Set Up

The goal of this research is to propose a novel method of achieving domain adaptation when facing distribution shift between the source and the target environment.  

Suppose we are given a causal DAG which involves Y (the target variable), V (a set of predictors of Y, all of which are causal ancesters of Y), and possibly U (a set of unobserved variables in the DAG). We further assume that the DAG factorization across the two environments are the same, but some of the terms of the factorization may be different.

We further assume that the information of which edge has shifted is given to us. That is, we do not consider how to detect distribution shift in this research. 

We have a large amount of training data available to us in the training environment ($N$). In out problem, we consider the case where we have a few data points ($M$) in the shifted environment, where $N >> M$. 

The task is to train a predictive model that achieves the highest accuracy in the shifted environment.

## III. Background

#### Definition of stability and distribution shift (Subbaswamy et al. in "unifying framework"):
We graphically charaterize instability in terms of edges in the graph of the data generating process. Assume that there is a set of environments such that a predictive problem is mapped to the same graph structure $G$. However, each environment is a different instantiation of that graph such that certain mechanisms differ. Thus, the factorization of the data distribution is the same in each environment, but terms in the factorization corresponding to shifts will vary across environments. 


#### Defintion of unstable edge (modified from Subbaswamy et al. in "unifying framework"):
An edge $X \to Y$ is said to be unstable if, in 2 different environments of $G$, the distribution $P(Y(x', V(x)) - Y(x))$ changes, where:

- $V = pa(Y) \setminus X$

- $Y(x)$ is the counterfactual value of $Y$ had $X$ been $x$

- $Y(x', V(x))$ is the counterfactual value of $Y$ had $X$ been $x'$ and had $V$ been counterfactually generated under $X = x$.

#### Using unstable edge to model common types of distribution shifts: 

The above definition is able to model:

1. Edge-strength shift: An edge strength shift in edge $X \to Y$ corresponds to a change in the natural direct effect: for $V = pa(Y) \setminus X$, $E(Y(x', V(x)) - Y(x))$ changes. 

Notice that when this expected value changes, the distribution $P(Y(x', V(x)) - Y(x))$ necessarily changes too. Thus, our definition of unstable edge is able to capture edge-strength shift.

2. Mechanism shift: A shift in the mechanism generating a variable $Y$ corresponds to arbitrary changes in the distribution $P(Y∣pa(Y))$. Suppose $Y = f(pa(Y)) + \epsilon$ is the deterministic data generating process of $Y$ given its parents, where $f$ is a function representing the relationship between $Y$ and its parents $pa(Y)$, and $\epsilon$ is an error term that is independent of $pa(Y)$.

Now, let's consider a mechanism shift in the edges $pa(Y) \to Y$. If the function $f$ changes to $f'$, the data-generating process becomes $Y = f'(pa(Y)) + \epsilon$. This change in the mechanism affects the distribution $P(Y∣pa(Y))$. 

Suppose $X \in pa(Y)$ and $V = pa(Y) \setminus X$. Let's examine the edge $X\to Y$ first. Under the counterfactual scenarios $Y(x)$ and $Y(x', V(x))$, the values of $Y$ are determined by the respective functions $f$ and $f'$ before and after the mechanism shift, which means the difference in counterfactual values, $Y(x', V(x)) - Y(x)$, will be influenced by this mechanism shift.

Since the mechanism shift can lead to arbitrary changes in the distribution $P(Y∣pa(Y))$, it can also cause changes in the distribution $P(Y(x', V(x)) - Y(x))$. Thus, our definition of unstable edge is able to capture the case when mechanism shift happens. 

In fact, when there are arbitrary changes in the distribution $P(Y∣pa(Y))$, it is likely that, for each $X_i \in pa(Y)$ and $V_i = pa(Y) \setminus X_i$, the distributions $P(Y(x_i', V_i(x_i))) - Y(x_i))$ changes. Thus, we should mark all edges coming into $Y$ as unstable.

#### Generalized Additive Model

A Generalized Additive Model (GAM) takes the form

$ g(\operatorname {E}(Y))=\beta _{0}+f_{1}(x_{1})+f_{2}(x_{2})+\cdots +f_{m}(x_{m}) $

where $f_i$ can be arbitrarily complex functions and $x_i$'s are the predictors of $Y$. 

The link function $g(\cdot)$ serves the following purposes:
1. it allows for the modeling of relationships between the predictor variables and the response variable that may not be linear, by transforming the expected value of the response variable.
2. it ensures that the expected value of the response variable lies within the permissible range, especially for response variables with bounded ranges (e.g., probabilities must be between 0 and 1).

Some common link functions used in GAMs include:

- Identity link: $g(E(Y)) = E(Y)$, used for continuous response variables in linear regression models.
- Logit link: $g(E(Y)) = \log\left(\frac{E(Y)}{1 - E(Y)}\right)$, used for binary response variables in logistic regression models.
- Log link: $g(E(Y)) = \log(E(Y))$, used for count or rate data in Poisson and negative binomial regression models.

The choice of link function depends on the distribution of the response variable and the desired relationship between the predictor variables and the response variable.


## IV. Finding stable component in a GAM using d-seperation rules

Suppose we are given a set of observed causal ancestors of $Y$, the causal relationship between them and possibly some unobserved variables, and which edges in the DAG are unstable across the source and target environments. Now, if $Y$ can be modeled using a GAM, how can we determine which variables are stable across the two environments?

#### Definition of a stable predictor

A predictor is stable if the association between such predictor and $Y$ are constant across two environments, given all the other variables that are also in the model. In other words, a predictor is stable if the portion of its effect to $Y$ that does not go through any other predictors is constant. **Mathematically ??**

#### Definition of a stable component

A stable component of a GAM across the source and target environment is a collection/set of stable predictors. 

#### Finding the stable component

The following algorithm outputs the set of predictors that forms a stable component of a GAM. 

Step 1: draw a square around all the predictors in the model to indicate that, when training a predictive model on $Y$ using the predictors $X_1, X_2, ..., X_n$, it is analogous to conditioning on them in the DAG. This is because, in the context of a DAG, conditioning on variables means considering their values in the analysis, which is what the machine learning model does when using them as predictors.

Step 2: for each predictor in the ML model, check all the paths from that variable to Y, including both front-door and back-door paths. A predictor is stable if there is no unstable path from that predictor to $Y$, when conditioning on the rest of the predictors. 

The set of all the stable predictors found in step 2 is the stable component that is directly transferable from the source to the target environment.

#### Example

Suppose we are given the following data generating process, where $U1$ and $U2$ are unobserved and unstable edges across the two environments are colored red. 

<img src="example_1_DAG.png" alt="pic" width="30%">

To determine a stable component of a GAM that predicts $Y$ using $X_1, X_2, X_3$, we need to draw a square around all the predictors, see below. By d-separation, when learning a ML model that predicts $Y$ using $X_1, X_2, X_3$, the association between $X_1$ and $Y$ is learned through the path $X_1 \to U_2 \to Y$ only, because the other flow of association from $X_1$ to $Y$ that goes through $X_2$ is blocked since $X_2$ is being "conditioned on" during the model training process. Note: this is because -- when learning the association between $X_1$ and $Y$, we're essentially asking the question: how does $Y$ change depending on $X_1$, given all else constant. The "given all else constant" part is equivalent to "conditioning" in causal terms, because we are fixing $X_2$ and $X_3$ to some specific values when learning the relationship between $X_1$ and $Y$. 

**Question: is that still true when $X_1$ and $X_2$ interact?**

<img src="example_1_conditioned.png" alt="pic" width="30%">

Therefore, if we learned a model $g(\operatorname {E}(Y))=\beta _{0}+f_{1}(X_{1})+f_{2}(X_{2}) +f_{3}(X_{3}) $ that is able to predict $Y$ in the source environment, then $f_1(X_1)$ and $f_3(X_3)$ are stable predictors across the two environments whereas $f_2(X_2)$ is not stable because there is an unstable path between $X_2$ and $Y$. So, the association learned in the source environment $f_2(X_2)$ is not transportable.


## V. Identifying and dealing with interaction terms

In the above example, when $X_1$ and $X_2$ interact, then we are not able to accurately model $Y$ with a fully additive model. When $X_1$ and $X_2$ interact, the association between $X_1, X_2$ and $Y$ can no longer be modeled accurately using $Y = f_{1}(X_{1})+f_{2}(X_{2}) + \text{intercept}$. Instead, we have to model such association using $Y = f(X_1, X_2) + \text{intercept}$.

Therefore, we need to add one more criteria on the definition of "stable component" of a GAM across the source and target environments. Specifically, we need to require that **non of the predictors in the stable component interact with the predictors in the unstable component** (defined by the set difference between all predictors and the predictors in the stable componenet).

THOUGHTS: MAYBE WE CAN JUST BORROW AN EXISTING STATISTICAL TEST TO DETERMINE WHETHER THERE EXISTS AN INTERACTION TERM OR NOT.

#### Algorithm for finding the largest subset of stable component (as determined by the d-sep rules) that does not interact with the unstable component:

Step 1: create two neural nets, one for the initially identified stable component (call it $N_a$) and another for the unstable component (call it $N_b$). Let the outputs of the two neural nets be $Y_1$ and $Y_2$ respectively. Let the prediction $\hat{Y} = g(Y_1 + Y_2)$ where $g$ is some pre-defined link function. 

Step 2: co-train the two models with the target of minimizing the difference (e.g. MSE / cross-entropy loss) between $\hat{Y}$ and the ground truth $Y$. Stop until training converges.

Step 3:

```
While True:

    interaction_detected = False
    
    for p in N_a.predictors():
        create connections from p to nodes in the second layer of N_b
        continue training until convergence
        
        if at least one of these newly created edge has weights > threshold:
            # this means p interact with at least one predictor in N_b
            interaction_detected = True
            
            remove p from N_a
            add p as a new predictor in N_b
            retrain N_a and N_b simultaneously until convergence
            
            break out of the for loop
     
     if interaction_detected == False:
         break out of the while loop
```

Now, the predictors left in $N_a$ is the stable component that does not interact with:
1. any predictor in the initially detected unstable component
2. any predictor that interacts with predictors in the initially detected unstable component

#### Note: 

This paper on prior NN might be relevant: http://proceedings.mlr.press/v119/rieger20a/rieger20a.pdf

## VI. Experiment 1: my own DGP, 3 predictors, no interaction

In [157]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import r2_score
import copy

In [264]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

    
def train_single_net(net, inputs, targets, num_epochs=1000):
    '''
    Trains a single neural network using the provided inputs and targets.

    Parameters:
    net (nn.Module): The neural network to train.
    inputs (torch.Tensor): The input features tensor of shape (n_samples, n_features).
    targets (torch.Tensor): The ground truth target variable tensor of shape (n_samples, 1).
    num_epochs (int, optional): The number of epochs to train the neural network. Default is 1000.
    learning_rate (float, optional): The learning rate for the optimizer. Default is 0.001.

    Returns:
    nn.Module: The trained neural network.
    '''

    optimizer = optim.Adam(net.parameters())
    criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    return net


def co_train(net_a, net_b, inputs, targets, net_a_predictors, net_b_predictors, link_function, num_epochs=1000):
    '''
    Co-trains two neural networks, net_a and net_b, with the target of minimizing the 
    loss between the predicted output (Y_hat = link_function(net_a_output + net_b_output)) 
    and the ground truth.

    Parameters:
    net_a (nn.Module): The neural network representing the stable component.
    net_b (nn.Module): The neural network representing the unstable component.
    inputs (torch.Tensor): The input features tensor of shape (n_samples, n_features).
    targets (torch.Tensor): The ground truth target variable tensor of shape (n_samples, 1).
    net_a_predictors (list): The indices of input features in 'inputs' that are part of net_a.
    link_function (callable): A function that combines the outputs of net_a and net_b.
    num_epochs (int, optional): The number of epochs to train the neural networks. Default is 10000.

    Returns:
    tuple: The co-trained neural networks net_a and net_b.
    '''
    
    optimizer_a = optim.Adam(net_a.parameters())
    optimizer_b = optim.Adam(net_b.parameters())
    criterion = nn.MSELoss() # add a loss to prior?!!!?

    for _ in range(num_epochs):
        outputs_a = net_a(inputs[:, net_a_predictors])
        outputs_b = net_b(inputs[:, net_b_predictors])
        y_hat = link_function(outputs_a + outputs_b)

        loss = criterion(y_hat, targets)
        
        optimizer_a.zero_grad()
        optimizer_b.zero_grad()

        loss.backward()

        optimizer_a.step()
        optimizer_b.step()

    return net_a, net_b

# This is probabily incorrect. Need to test it. 
def largest_stable_subset(net_a, 
                          net_b, 
                          inputs, 
                          targets, 
                          net_a_predictors, 
                          net_b_predictors, 
                          link_function, 
                          threshold, 
                          num_epochs):
    
    while True:
        interaction_detected = False
        net_a_copy = copy.deepcopy(net_a)
        net_b_copy = copy.deepcopy(net_b)

        for predictor in net_a_predictors:
            # Append the new predictor/feature index to net_b_predictors
            net_b_predictors.append(predictor)
            
            # Create a copy of the original net_b.layer1.weight
            original_weight = net_b.layer1.weight.clone()

            # Add a column of zeros for the new connection
            zero_column = torch.zeros(net_b.layer1.weight.size(0), 1)
            new_weight = nn.Parameter(torch.cat((zero_column, original_weight), dim=1))
            
            # Update the weights between the 1st and 2nd layer of net_b, 
            # since net_b now has one more input neuron
            input_dim = net_b.layer1.in_features + 1
            output_dim = net_b.layer1.out_features
            net_b.layer1 = nn.Linear(input_dim, output_dim)
            net_b.layer1.weight = new_weight
            
            # Continue training until convergence
            net_a, net_b = co_train(net_a=net_a, 
                                    net_b=net_b, 
                                    inputs=inputs, 
                                    targets=targets, 
                                    net_a_predictors=net_a_predictors,
                                    net_b_predictors=net_b_predictors,
                                    link_function=link_function,
                                    num_epochs=num_epochs)
            
            # The newly added connects are the weights in the first column of net_b.layer1.weight
            new_connections_after_training = net_b.layer1.weight[:, 0]
            print("new connections weights", new_connections_after_training)
            print("net a layer 1 weight: ", net_a.layer1.weight)
            print("net b layer 1 weight: ", net_b.layer1.weight)
            
            # Check if at least one newly created edge has weights > threshold
            if (new_connections_after_training > threshold).any():
                # If there is interaction
                
                interaction_detected = True

                # Change the input dim of net_a
                input_dim = net_a.layer1.in_features - 1
                output_dim = net_a.layer1.out_features
                net_a.layer1 = nn.Linear(input_dim, output_dim)
                
                # 2) remove the predictor index from net_a_predictors
                net_a_predictors.remove(predictor)

                # Retrain net_a and net_b simultaneously until convergence
                net_a, net_b = co_train(net_a=net_a, 
                                        net_b=net_b, 
                                        inputs=inputs, 
                                        targets=targets, 
                                        net_a_predictors=net_a_predictors,
                                        net_b_predictors=net_b_predictors,
                                        link_function=link_function,
                                        num_epochs=num_epochs)
                break

            else:
                # If there's no interaction:
                
                # 1) revert net_a and net_b to its original state
                net_a = net_a_copy
                net_b = net_b_copy
                
#                 input_dim = net_b.layer1.in_features - 1
#                 output_dim = net_b.layer1.out_features
#                 net_b.layer1 = nn.Linear(input_dim, output_dim)
#                 net_b.layer1.weight = original_weight
                
                # 2) remove the new predictor index from net_b_predictors
                net_b_predictors.remove(predictor)
                
        if not interaction_detected:
            break

    return net_a, net_b



In [496]:
def data_generator(strength=1, size=1000):
    '''
    DGP based on the DAG in section IV
    '''
    
    x1 = np.random.normal(0, 1, size=size)
    x3 = np.random.normal(0, 1, size=size)
    
    x2 = strength*x1 + 10 * (np.random.normal(0, 1, size=size))**3 #0.1*strength*x1 + 3*np.random.normal(0, 1, size=size)
    
    u1 = strength*x2 + 3*x3 + np.random.normal(0, 1, size=size)
    u2 = 5*x1 + np.random.normal(0, 1, size=size)

    y = 3*u1 + u2**2 + np.exp(x3) + np.random.normal(0, 1, size=size)

    return x1, x2, x3, y

# y = strength*x2 + 3*x3 + np.random.normal(0, 1, size=size) + 5*x1 + np.random.normal(0, 1, size=size) + x3 + np.random.normal(0, 1, size=size)
# y = (strength*0.3*strength + 5) * x1 + 4*x3 + np.random.normal(0, 1, size=size) + np.random.normal(0, 1, size=size) + np.random.normal(0, 1, size=size)

    
def get_data(sample_size, strength=1):
    X1, X2, X3, Y = data_generator(strength=strength, size=sample_size)
    Xmat = np.hstack((X1.reshape(-1, 1), X2.reshape(-1, 1), X3.reshape(-1, 1)))
    return Xmat, Y

In [497]:
# Get Training and Test Data
sample_size = 10000
Xmat, Y = get_data(sample_size, strength=2)
Xmat_test, Y_test = get_data(sample_size, strength=2)

# Convert the input features and target variable to tensors
inputs = torch.tensor(Xmat, dtype=torch.float32)
targets = torch.tensor(Y, dtype=torch.float32).unsqueeze(1)

inputs_test = torch.tensor(Xmat_test, dtype=torch.float32)
targets_test = torch.tensor(Y_test, dtype=torch.float32).unsqueeze(1)

In [498]:
# Create neural networks net_a and net_b
input_size_a = 2
input_size_b = 1
hidden_size = 10
output_size = 1

net_a = NeuralNet(input_size_a, hidden_size, output_size)
net_b = NeuralNet(input_size_b, hidden_size, output_size)

# Define the link function as the identity function
link_function = torch.nn.Identity()

In [499]:
# Co-train net_a and net_b
net_a_predictors = [0, 2]
net_b_predictors = [i for i in range(inputs.shape[1]) if i not in net_a_predictors]
    
net_a, net_b = co_train(net_a=net_a, 
                        net_b=net_b, 
                        inputs=inputs, 
                        targets=targets, 
                        net_a_predictors=net_a_predictors, 
                        net_b_predictors=net_b_predictors,
                        link_function=link_function,
                        num_epochs=10000)

In [500]:
# Get the weights of the neural network
print("net b layer 1 weight: ", net_b.layer1.weight)
print()
for name, param in net_b.named_parameters():
    if param.requires_grad:
        print(name, param.data)

net b layer 1 weight:  Parameter containing:
tensor([[-1.5175],
        [-1.0549],
        [ 1.4286],
        [-1.9180],
        [-1.1677],
        [-0.8985],
        [-0.4877],
        [ 1.8959],
        [ 1.5812],
        [-1.5684]], requires_grad=True)

layer1.weight tensor([[-1.5175],
        [-1.0549],
        [ 1.4286],
        [-1.9180],
        [-1.1677],
        [-0.8985],
        [-0.4877],
        [ 1.8959],
        [ 1.5812],
        [-1.5684]])
layer1.bias tensor([ 2.4323, -0.5087, -3.6505,  1.9652,  0.3322,  4.0892,  3.1666, -0.6447,
        -2.7617,  1.8367])
layer2.weight tensor([[-0.6852, -0.3825,  0.9988, -0.8597, -0.6320, -0.5116, -0.4012,  1.3917,
          1.2194, -0.9714]])
layer2.bias tensor([-0.9006])


In [501]:
n_features = inputs.shape[1]
net_b_predictors = [i for i in range(n_features) if i not in net_a_predictors]

# Get the outputs of net_a and net_b
outputs_a = net_a(inputs_test[:, net_a_predictors]).detach().numpy()
outputs_b = net_b(inputs_test[:, net_b_predictors]).detach().numpy()

# Get the combined output of the full model
combined_output = link_function(net_a(inputs_test[:, net_a_predictors]) + net_b(inputs_test[:, net_b_predictors])).detach().numpy()

# Calculate R^2 scores
r2_net_a = r2_score(targets_test.numpy(), outputs_a)
r2_net_b = r2_score(targets_test.numpy(), outputs_b)
r2_full_model = r2_score(targets_test.numpy(), combined_output)

print(f"R^2 score for net_a: {r2_net_a}")
print(f"R^2 score for net_b: {r2_net_b}")
print(f"R^2 score for the full model: {r2_full_model}")

R^2 score for net_a: 0.02010162508761726
R^2 score for net_b: 0.9512457839609639
R^2 score for the full model: 0.997947279374086


In [152]:
largest_stable_subset(net_a=net_a, 
                      net_b=net_b, 
                      inputs=inputs, 
                      targets=targets, 
                      net_a_predictors=net_a_predictors, 
                      net_b_predictors=net_b_predictors, 
                      link_function=link_function, 
                      threshold=0.2, 
                      num_epochs=5000)

new connections weights tensor([-0.2180, -0.1939, -0.1232], grad_fn=<SelectBackward0>)
net a layer 1 weight:  Parameter containing:
tensor([[ 0.0765, -3.6594],
        [-7.2253,  1.8693],
        [ 6.5555,  1.6924]], requires_grad=True)
net b layer 1 weight:  Parameter containing:
tensor([[-0.2180, -0.0037],
        [-0.1939,  0.0141],
        [-0.1232, -0.1274]], requires_grad=True)
new connections weights tensor([ 0.0000, -0.1833, -0.3093], grad_fn=<SelectBackward0>)
net a layer 1 weight:  Parameter containing:
tensor([[ 0.0095, -3.9409],
        [-7.1439,  1.7159],
        [ 6.4215,  1.5799]], requires_grad=True)
net b layer 1 weight:  Parameter containing:
tensor([[ 0.0000,  0.0838],
        [-0.1833,  0.8797],
        [-0.3093, -0.9395]], requires_grad=True)


(NeuralNet(
   (layer1): Linear(in_features=2, out_features=3, bias=True)
   (layer2): Linear(in_features=3, out_features=1, bias=True)
 ),
 NeuralNet(
   (layer1): Linear(in_features=2, out_features=3, bias=True)
   (layer2): Linear(in_features=3, out_features=1, bias=True)
 ))

In [389]:
net_b_predictors

[1]

In [524]:
# Apply distribution shift
# Get Training and Test Data
Xmat_target, Y_target = get_data(50, strength=0.2)
Xmat_test_target, Y_test_target = get_data(10000, strength=0.2)

# Convert the input features and target variable to tensors
inputs_target = torch.tensor(Xmat_target, dtype=torch.float32)
targets_target = torch.tensor(Y_target, dtype=torch.float32).unsqueeze(1)

inputs_test_target = torch.tensor(Xmat_test_target, dtype=torch.float32)
targets_test_target = torch.tensor(Y_test_target, dtype=torch.float32).unsqueeze(1)


### Our method ###
net_a_target = copy.deepcopy(net_a)

outputs_a_training = net_a_target(inputs_target[:, net_a_predictors]).detach().numpy()

net_b_target = train_single_net(copy.deepcopy(net_b), 
                                inputs_target[:, net_b_predictors], 
                                targets_target - outputs_a_training, 
                                num_epochs=10000)


# Get the outputs of net_a_target and net_b_target
outputs_a = net_a_target(inputs_test_target[:, net_a_predictors]).detach().numpy()
outputs_b = net_b_target(inputs_test_target[:, net_b_predictors]).detach().numpy()

# Get the combined output of the full model
combined_output = link_function(outputs_a + outputs_b)

# a = outputs_a
# c = combined_output

r2_our_method = r2_score(targets_test_target.numpy(), combined_output)



### Baseline 1: use source model directly on the target environment ###
net_a_target = copy.deepcopy(net_a)
net_b_target = copy.deepcopy(net_b)

# Get the outputs of net_a_target and net_b_target
outputs_a = net_a_target(inputs_test_target[:, net_a_predictors]).detach().numpy()
outputs_b = net_b_target(inputs_test_target[:, net_b_predictors]).detach().numpy()

# Get the combined output of the full model
combined_output = link_function(outputs_a + outputs_b)

r2_baseline_1 = r2_score(targets_test_target.numpy(), combined_output)

print(len(inputs_target))

### Baseline 2: directly retrain a new model ###

# Create neural networks net_a and net_b
input_size_a = 2
input_size_b = 1
hidden_size = 10
output_size = 1

net_a_baseline_2 = NeuralNet(input_size_a, hidden_size, output_size)
net_b_baseline_2 = NeuralNet(input_size_b, hidden_size, output_size)

net_a_target, net_b_target = co_train(net_a=net_a_baseline_2, 
                                      net_b=net_b_baseline_2, 
                                      inputs=inputs_target, 
                                      targets=targets_target, 
                                      net_a_predictors=net_a_predictors, 
                                      net_b_predictors=net_b_predictors,
                                      link_function=link_function,
                                      num_epochs=12000)

# Get the outputs of net_a_target and net_b_target
outputs_a = net_a_target(inputs_test_target[:, net_a_predictors]).detach().numpy()
outputs_b = net_b_target(inputs_test_target[:, net_b_predictors]).detach().numpy()

# Get the combined output of the full model
combined_output = link_function(outputs_a + outputs_b)
r2_baseline_2 = r2_score(targets_test_target.numpy(), combined_output)

# a2 = outputs_a
# c2 = combined_output


print(f"R^2 score for our method: {r2_our_method}")
print(f"R^2 score for baseline 1: {r2_baseline_1}")
print(f"R^2 score for baseline 2: {r2_baseline_2}")

50
R^2 score for our method: 0.9335891298003772
R^2 score for baseline 1: -20.54486319434331
R^2 score for baseline 2: 0.8738260197651074


In [418]:
the strength of x1 -> x2 matters a lot, because if x1 is too predictive of x2,
then the initial net_a and net_b might learn the alternative: really strong net_a that only uses x1, and a trash ent_b that doesn't really use x2.

array([[ 32.797195],
       [ 46.679672],
       [-16.292498],
       ...,
       [ 19.985012],
       [ 21.911715],
       [ 23.590187]], dtype=float32)

In [419]:
a2

array([[35.070965 ],
       [73.17574  ],
       [ 0.7389033],
       ...,
       [29.161621 ],
       [26.557375 ],
       [30.938162 ]], dtype=float32)

In [382]:
c

array([[68.49792   ],
       [ 5.8692274 ],
       [-0.48029232],
       ...,
       [15.224551  ],
       [59.207523  ],
       [94.538574  ]], dtype=float32)

In [383]:
c2

array([[65.710106 ],
       [ 3.846383 ],
       [-3.4544182],
       ...,
       [ 9.74332  ],
       [56.34586  ],
       [97.97184  ]], dtype=float32)

In [289]:
# Apply distribution shift
# Get Training and Test Data
Xmat_target, Y_target = get_data(100, strength=1)
Xmat_test_target, Y_test_target = get_data(10000, strength=1)

# Convert the input features and target variable to tensors
inputs_train = torch.tensor(Xmat_target, dtype=torch.float32)
targets_train = torch.tensor(Y_target, dtype=torch.float32).unsqueeze(1)

inputs_test = torch.tensor(Xmat_test_target, dtype=torch.float32)
targets_test = torch.tensor(Y_test_target, dtype=torch.float32).unsqueeze(1)

print(net_a)
print("net a layer 1 weight: ", net_a.layer1.weight)

### Our method ###
net_a_target = copy.deepcopy(net_a)

outputs_a_training = net_a_target(inputs_train[:, net_a_predictors]).detach().numpy()

net_b_target = train_single_net(copy.deepcopy(net_b), 
                                inputs_train[:, net_b_predictors], 
                                targets_train - outputs_a_training, 
                                num_epochs=5000)


# Get the outputs of net_a_target and net_b_target
outputs_a = net_a_target(inputs_test[:, net_a_predictors]).detach().numpy()
outputs_b = net_b_target(inputs_test[:, net_b_predictors]).detach().numpy()

# Get the combined output of the full model
combined_output = link_function(outputs_a + outputs_b)

r2_our_method = r2_score(targets_test.numpy(), combined_output)


NeuralNet(
  (layer1): Linear(in_features=2, out_features=3, bias=True)
  (layer2): Linear(in_features=3, out_features=1, bias=True)
)
net a layer 1 weight:  Parameter containing:
tensor([[-2.3780,  0.0166],
        [-1.2359,  1.7689],
        [ 2.9769, -0.0293]], requires_grad=True)


In [290]:
r2_our_method

0.8684668918672309

## VII. Experiment 2


questions:

1. **If two variables do no interact, is it necessary that they are additive?**
1. is it possible to prove that, if there are some edges into Y that are unstable and some edges into Y that are stable, then there must be no interaction between the unstable variables and the stable variables (cuz otherwise these "stable" variables would have been also unstable)? Can we then conclude that their relationship must be additive?

GPT answer: 
It is not necessarily true that if there are some unstable edges into $Y$ and some stable edges into $Y$, there must be no interaction between the unstable variables and the stable variables. The stability of an edge depends on the consistency of the relationship between the variables across different environments. It is possible for variables with stable edges to interact with variables having unstable edges, but the nature of the interaction might be stable across environments, making the stable variables' edges remain stable.

e.g.: Y = 3x + 4y + 5xy. If under shift, Y = 3x + 10y + 5xy, then x->Y is stable but y->Y is unstable, while x and y have interactions!

## Appendix 1. Other Approaches That I Have Tried

## Next steps:

- Is it correct that the association learned by net_a and net_b are exactly through the association paths determined by d-speration? How can we prove it? PROVE IT: do some probabilities and drop the terms by indepedent (d-sep), eventually we will show that the association learned by net_a / net_b are exactly the things we wanted to learn. 

- More baseline models

- Figure out the interaction terms thing.

- Test on some real dataset

### Summer:

Maybe it's a good idea to take a step back and 1) learn causal inference thoroughly and systematically, and 2) read many papers related to domain adaptation and causal inference. Need to see what people are working on first. 

Ideally, find a topic that is 1) based on theory, not just emperical, and 2) have connects with this current piece of work.




## Biggest Limitations of this research
- we have to know the causal DAG
- we have to require/assume that Y follows an additive Data Generating Process
- we have to know which edge has shifted between two environments